In [120]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from indicadores import *
import labeling as lb

In [132]:
def prepare_data(olhc):
    # Calculando os indicadores e normalizando-os
    data = agg_indicators(olhc)
    data = normalize_indicators(data)

    # Rotulando os dados
    y = np.array(lb.labelData(olhc, 0.1)).ravel()

    # Eliminando as linhas com NaN
    data["y"] = y
    data = data.dropna()

    # Convertendo para numpy arrays, caso ainda não estejam
    X = np.array(data)[:, :-1]
    y = np.array(data)[:, -1]

    # # Divide os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

## Fazendo a Rede Neural

In [136]:
def mlp(olhc, hidden_layers=(100, 100, 100), activation='logistic', 
        solver='adam', max_iter=500, random_state=42):
    """
    Treina um modelo de rede neural MLP e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame contendo os dados de entrada.
    - hidden_layers: Tupla com o tamanho das camadas ocultas.
    - activation: Função de ativação a ser usada.
    - solver: Algoritmo de otimização a ser usado.
    - max_iter: Número máximo de iterações.
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_mlp: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    # Preparando os dados
    X_train, X_test, y_train, y_test = prepare_data(olhc)

    # Definindo a rede neural com múltiplas camadas
    mlp = MLPClassifier(hidden_layer_sizes = hidden_layers,
                        activation = activation,
                        solver = solver,
                        max_iter = max_iter,
                        random_state = random_state)

    # Treina a rede neural
    mlp.fit(X_train, y_train)

    # Faz previsões de classe
    y_pred_mlp = mlp.predict(X_test)

    # Exibe o relatório de classificação para o MLP
    report = classification_report(y_test, y_pred_mlp)
    print(report)

    return y_pred_mlp

In [138]:
y_pred = mlp(tsla_data)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         5
         0.0       0.48      0.32      0.38       328
         1.0       0.53      0.69      0.60       365

    accuracy                           0.51       698
   macro avg       0.33      0.34      0.33       698
weighted avg       0.50      0.51      0.49       698



c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Fazendo o modelo de Random Forest

In [140]:
def random_forest(olhc, n_estimators=100, max_depth=None, random_state=42):
    """
    Treina um modelo de Random Forest e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame com as features e a variável alvo.
    - target_column: Nome da coluna alvo no DataFrame.
    - n_estimators: Número de árvores na floresta.
    - max_depth: Profundidade máxima das árvores (None para ilimitado).
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_rf: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    # Preparando os dados
    X_train, X_test, y_train, y_test = prepare_data(olhc)

    # Definindo o modelo Random Forest
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, 
                                 random_state=random_state)

    # Treina o modelo Random Forest
    rf.fit(X_train, y_train)

    # Faz previsões de classe
    y_pred_rf = rf.predict(X_test)

    # Exibe o relatório de classificação para o Random Forest
    report = classification_report(y_test, y_pred_rf)
    print(report)

    return y_pred_rf

In [141]:
y_pred = random_forest(tsla_data)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         5
         0.0       0.49      0.57      0.53       328
         1.0       0.54      0.48      0.51       365

    accuracy                           0.52       698
   macro avg       0.35      0.35      0.34       698
weighted avg       0.52      0.52      0.51       698



c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Fazendo o Modelo Gradient Boosting

In [142]:
def gradient_boosting(olhc, random_state=42):
    """
    Treina um modelo de Gradient Boosting e retorna as previsões e o relatório de classificação.

    Parâmetros:
    - data: DataFrame com as features e a variável alvo.
    - target_column: Nome da coluna alvo no DataFrame.
    - random_state: Semente para a geração de números aleatórios.

    Retorna:
    - y_pred_gb: Previsões das classes no conjunto de teste.
    - report: Relatório de classificação.
    """
    # Preparando os dados
    X_train, X_test, y_train, y_test = prepare_data(olhc)

    # Definindo o modelo Gradient Boosting
    gb_model = GradientBoostingClassifier(random_state=random_state)

    # Treina o modelo Gradient Boosting
    gb_model.fit(X_train, y_train)

    # Faz previsões de classe
    y_pred_gb = gb_model.predict(X_test)

    # Exibe o relatório de classificação para o Gradient Boosting
    report = classification_report(y_test, y_pred_gb)
    print(report)

    return y_pred_gb

In [143]:
y_pred = gradient_boosting(tsla_data)

c:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\site-packages\ta\trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         5
         0.0       0.48      0.54      0.51       328
         1.0       0.53      0.48      0.51       365

    accuracy                           0.50       698
   macro avg       0.34      0.34      0.34       698
weighted avg       0.51      0.50      0.50       698



In [56]:
# # Faz previsões de probabilidade
# y_pred_probs = rf.predict_proba(X_test)

# # Identifica a classe com maior probabilidade
# y_pred_indices = np.argmax(y_pred_probs, axis=1)

# # Mapeia os índices para as classes de interesse
# class_mapping = {0: 0, 1: 1, 2: -1}  # ajuste conforme necessário para sua classificação
# y_pred = np.vectorize(class_mapping.get)(y_pred_indices)

# # Avalia o modelo
# print(classification_report(y_test, y_pred, target_names=["Desfazer Posição", "Compra", "Short"]))

                  precision    recall  f1-score   support

Desfazer Posição       0.01      0.78      0.02         9
          Compra       0.00      0.00      0.00       357
           Short       0.51      0.12      0.19       347

        accuracy                           0.07       713
       macro avg       0.17      0.30      0.07       713
    weighted avg       0.25      0.07      0.09       713



c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Léo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier